# Molecular Dynamics Tutorial

## Example Script

In [ ]:
from ase.io import read, Trajectory
from carmm.run.aims_path import set_aims_command
from carmm.run.aims_calculator import get_aims_calculator
import numpy as np
from ase.units import fs
from ase.md.npt import NPT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary, ZeroRotation

file = 'atoms.xyz'

basis = 'light'
xc = 'pbe'  # + MBD-NL
dim = 3
k_grid = (1, 1, 2)
spin = 'none'
ensemble = 'nvt'  # 'npt' or 'nvt'
temp = 298.15  # in K
equil_steps = 1000  # Number of steps for equilibration run
prod_steps = 3000  # Number of steps for production run
restart = False  # If True, set file = .traj (to get last frame of previous dyn)
hpc = 'hawk'  # 'hawk', 'hawk-amd', 'young', 'isambard'

jobname = f'{tsite}-{osite}-{ads}_{temp}-{ensemble}'


atoms = read(file)

# Set up Aims calculator (Standard)
set_aims_command(hpc=hpc, basis_set=basis, defaults=2020)
fhi_calc = get_aims_calculator(dimensions=dim, xc=xc,
                               compute_analytical_stress=True, k_grid=k_grid)
fhi_calc.set(
    many_body_dispersion_nl=' ',
    spin=spin,
    relativistic=('atomic_zora', 'scalar'),
    sc_accuracy_etot=1e-6,
    sc_accuracy_eev=1e-3,
    sc_accuracy_rho=1e-6,
    sc_accuracy_forces=1e-4,
)
atoms.set_calculator(fhi_calc)

# Dynamics setup
externalstress = atoms.get_stress()

if ensemble == 'npt':
    pfactor = (50*fs)**2 * 0.7536243659298125  # pfactor = ptime^2 * Bulk modulus (Calc Bulk mod. using EOS, units of eV/A^3)
if ensemble == 'nvt':
    pfactor = None  # Turns off barostat
else:
    raise ValueError('ensemble not recognised/implemented')
    
if not restart:
    # Initialisation
    MaxwellBoltzmannDistribution(atoms, temperature_K=temp)  # Initialise atomic velocities as Boltzmann distribution at desired temperature
    Stationary(atoms)  # Center-of-mass momentum set to zero (unit cell stationary overall)
    ZeroRotation(atoms)  # Total angular momentum set to zero (unit cell not rotating overall)

    # Equilibration (only required for new simulation)
    dyn = NPT(atoms,
              timestep=1*fs,
              temperature_K=temp,
              externalstress=externalstress,
              ttime=30*fs,  # Thermostat timestep
              pfactor=pfactor,  # Barostat timestep -> pfactor
              mask=np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]))
    traj = Trajectory(f'{jobname}_equil.traj', 'w', atoms)
    dyn.attach(traj, interval=2)
    dyn.run(equil_steps)

# Production
dyn = NPT(atoms,
          timestep=1*fs,
          temperature_K=temp,
          externalstress=externalstress,
          ttime=30*fs,  # Thermostat timestep
          pfactor=pfactor,  # Barostat timestep -> pfactor
          mask=np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]))
traj = Trajectory(f'{jobname}_prod.traj', 'w', atoms)
dyn.attach(traj, interval=2)
dyn.run(prod_steps)

## Best Practices

Molecular dynamics can be more of an art than a science. That being said, here are some suggestions.

### Workflow

#### Summary

NPT (equilibration) -> NPT (production) -> NVT (equilibration) -> NVT (production)

NPT should be done when you have a new unit cell. When you are running dynamics on a similar system (*e.g.,* same bulk but new adsorbate) and already have its thermally expanded unit cell, you can skip NPT.
If you are running a simulation at a new temperature, you must always start with NPT to find the new thermally expanded unit cell.

#### Ensembles

Target ensemble is usually NVT, but for periodic systems you need to account for thermal expansion of the unit cell with respect to your 0 K optimised structure. Therefore, you first need to run NPT dynamics (Constant number of particles, pressure, and temperature) to find the average volume at the specified temperature. This could be done by averaging the lattice parameters over the production run (check that these produce a volume close to that of the average volume through the production run).

If your target ensemble is different or you are running on a molecular system, you must consider other properties of the system and other possible ensembles in a similar way to the above *i.e.,* find the values of the system that you're keeping constant if different to the 0 K value.

#### Equilibration -> Production

When you begin your simulation, although you may have initialised some of the macroscopic properties at their desired value, they will not necessarily stay that way immediately. Therefore, you must "equilibrate" the system. That is, allow the system to evolve for a time until: 1. The properties converge within a threshold of the desired values and 2. The system "forgets" its initial state sufficiently. The most common property to equilibrate is temperature, but volume/lattice parameters will also need to be equilibrated in NPT dynamics. There may also be other properties of your system that aren't correct initially. Plus it is good practice to allow the system to forget its starting structure to prevent unduly biasing your simulation.

You must define a threshold for equilibration of each of the properties. This could be a range in which the evolution of the property must stay for a defined length of time. *E.g.,* temperature must fluctuate within +/- 50 K of the desired system temperature for >1 fs.

The length of time you must allow the system to equilibrate for will depend on your system and properties. A higher temperature, for example, will generally take longer to equilibrate and may even require a looser convergence criterion *e.g.,* +/- 50 K over 1 fs may be okay for 298K, but +/- 100 K over 1 fs may be more appropriate for a 500 K simulation.

### Settings

You should test these yourself on your own system, but here are some general guidelines and advice.

#### Timestep

~1/10th of the time period of the fastest process of interest in your system. This will usually be the highest frequency vibration in your system. This is so that the dynamics is able to track this process appropriately.

Using the example of "fastest vibration", if the timestep is much larger than the time period, the vibrating atoms will travel too far with their current velocity before they are able to be pushed/pulled from the other (leading to the system breaking apart). The timestep being too small is generally the better option, however you needlessly increase the compuational cost of your calculation and reduce your possible total simulation time (which is already very short for *ab initio* MD!)

#### Choice of Thermo/Barostat

The Nosé-Hoover thermostat and barostat is generally advised for NVT/NPT dynamics, since it is deterministic (vs stochastic/random). In ASE, this is used through the NPT class.

Other thermo/barostats are available, particularly for other ensembles. Andersen dynamics is not advised, because the random decorrelation of velocities gives unphysical dynamics and so can't represent dynamic properties such as diffusion or viscocity. Berendsen dynamics is also not advised, because it doesn't formally result in proper NPT/NVT sampling, it is only "sufficiently good in practice (with a large tau_t and tau_p)."

#### Thermo/Barostat timesteps

This can generally be anything within reason, but should be tested on your system. Generally, it should be long enough so that you let the system evolve/relax at the new T/p, but short enough that it equilibrates appropriately quickly and that any rogue oscillations are killed quickly.